In [4]:
import cv2
import numpy as np
import glob
from skimage.measure import regionprops, label

In [11]:
def load_images(pattern):
    return glob.glob(pattern)

def process_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

    # Find contours of the particle
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)  # Assuming the particle is the largest contour

    # Sample 256 points from the contour
    sampled_contour = sample_contour_points(contour, 256)

    # Find centroid and the farthest point from the centroid
    M = cv2.moments(sampled_contour)
    centroid = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

    # Calculate radius based on the farthest point in the sampled contour
    radius = max(np.linalg.norm(sampled_contour - centroid, axis=1))

    # Create a mask for the ROI (circle around the centroid)
    mask = np.zeros_like(binary_image, dtype=np.uint8)
    cv2.circle(mask, centroid, int(radius), (255, 255, 255), -1)

    # Apply the mask to the binary image
    roi_image = cv2.bitwise_and(binary_image, mask)

    # Find orientation of the ROI
    labeled_img = label(roi_image)
    properties = regionprops(labeled_img)
    orientation = properties[0].orientation

    # Rotate the ROI image
    rotated_image = rotate_image(roi_image, centroid, -np.rad2deg(orientation))

    # Determine if image should be reversed
    if should_reverse(rotated_image, centroid):
        rotated_image = np.flipud(rotated_image)

    # Save or display the result
    cv2.imwrite('processed_' + image_path, rotated_image)

def sample_contour_points(contour, num_points):
    # Calculate the total perimeter and the interval for sampling
    total_perimeter = cv2.arcLength(contour, True)
    interval = total_perimeter / num_points

    sampled_points = []
    current_length = 0
    for i in range(len(contour)):
        if i == 0:
            sampled_points.append(contour[i][0])
        else:
            current_length += np.linalg.norm(contour[i][0] - contour[i - 1][0])
            if current_length >= interval:
                sampled_points.append(contour[i][0])
                current_length = 0

    # Convert the list of points to a numpy array
    return np.array(sampled_points, dtype=np.int32)

def rotate_image(image, center, angle):
    height, width = image.shape
    rot_matrix = cv2.getRotationMatrix2D(center, angle, 1)

    # Calculate the size of the new image
    new_width, new_height = width, height
    new_center = (new_width / 2, new_height / 2)

    new_rot_matrix = cv2.getRotationMatrix2D(new_center, angle, 1)
    new_rot_matrix[0, 2] += new_width / 2 - center[0]
    new_rot_matrix[1, 2] += new_height / 2 - center[1]

    # Rotate and fill empty space with white
    return cv2.warpAffine(image, new_rot_matrix, (new_width, new_height), borderValue=(255, 255, 255))

def should_reverse(image, centroid):
    height, width = image.shape
    x, y = centroid
    length_up = np.sum(image[:y, x] == 0)
    length_down = np.sum(image[y:, x] == 0)
    return length_down > length_up

In [12]:
def main():
    for image_path in load_images('*.png'):
        process_image(image_path)

if __name__ == '__main__':
    main()